In [ ]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

/opt/homebrew/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
! cp /kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d sartajbhuvaji/brain-tumor-classification-mri



mkdir: cannot create directory ‘/root/.kaggle’: File exists
 99% 86.0M/86.8M [00:00<00:00, 209MB/s]
100% 86.8M/86.8M [00:00<00:00, 195MB/s]


In [ ]:
!unzip "/content/brain-tumor-classification-mri.zip" -d "/home/MRI_images"

In [ ]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#Transforms
#pre-processing images 

def my_adjust_contrast():
    def _func(img):
        return  torchvision.transforms.functional.autocontrast(img)
    return _func 

def my_adjust_Sharpness():
    def _func(img):
        return  torchvision.transforms.functional.adjust_sharpness(img, 1) #increases sharpness by factor of 2
    return _func 

transformer=transforms.Compose([
    transforms.Resize((200,200)), 
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(           
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
        ),
    #my_adjust_Sharpness(),
    #my_adjust_contrast()
])


In [ ]:
#Dataloader
#"/Users/akshat/Desktop/ML/archive/Training/"
#Path for training and testing directory
train_path="/Users/akshat/Documents/archive(2)/Training/"
test_path="/Users/akshat/Documents/archive(2)/Testing/"

#loading the dataset using batch size 64 for train and 52 for test

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)



In [ ]:
classes = ['meningioma_tumor', 'no_tumor', 'pituitary_tumor']

In [ ]:
print(classes)

['meningioma_tumor', 'no_tumor', 'pituitary_tumor']


In [ ]:
#CNN Network

class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,stride=1,padding=1) #convolutional layer 1
        #Shape= (32,200,200)
        self.bn1=nn.BatchNorm2d(num_features=32)
        #Shape= (32,200,200)
        self.relu1=nn.ReLU()
        #Shape= (32,200,200)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (32,100,100)
        
        
        self.conv2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1) #convolutional layer 2
        #Shape= (64,100,100)
        self.relu2=nn.ReLU()
        #Shape= (64,100,100)
        
        
        
        self.conv3=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1) #convolutional layer 3
        #Shape= (128,100,100)
        self.bn3=nn.BatchNorm2d(num_features=128) 
        #Shape= (128,100,100)
        self.relu3=nn.ReLU() 
        #Shape= (128,100,100)
        
        self.conv4=nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1,padding=1) #convolutional layer 4
        #Shape= (256,100,100)
        self.relu4=nn.ReLU()
        #Shape= (256,100,100)
        
        #after 3 pooling, size would be reudced to (256,25,25)
        self.fc=nn.Linear(in_features=25 * 25 * 256,out_features= 1024) #fully connected layer 1
        self.fc2=nn.Linear(in_features=1024,out_features=64) ##fully connected layer 1
        self.fc3=nn.Linear(in_features=64,out_features=num_classes) ##fully connected layer 1
        #Feed forwad function
        
    #formwatd propogration
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)

        output=self.pool(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        
        output=self.pool(output) 
        
        output=self.conv4(output)
        output=self.relu4(output)
            
            
            
        output=output.view(-1,25*25*256) #flattening output
            
            
        #output=self.fc(output)
        output = self.relu1(self.fc(output)) 
        output = self.relu1(self.fc2(output))    
        output = self.fc3(output) 
            
        return output
            
        


In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
model=ConvNet(num_classes=4).to(device)

In [ ]:
#Adam Optmizer and cross entropy loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=15 #number of epochs the model is run on

In [ ]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [ ]:
print(train_count,test_count)

2044 294


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
best_accuracy=0.0
loss_train = []
train_acc = []
test_acc = []

#training the model
for epoch in range(num_epochs):
    
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=100 * train_accuracy/train_count
    train_loss=train_loss/train_count
    loss_train.append(train_loss)
    train_acc.append(train_accuracy)
    
    
    # testing the model
    model.eval()
    
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
            # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
            
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    test_accuracy = 100.0 * n_correct / n_samples
    test_acc.append(test_accuracy)
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))


    for i in range(3):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
    

Epoch: 0 Train Loss: tensor(1.9023) Train Accuracy: 48.82583170254403 Test Accuracy: 39.1156462585034
Accuracy of meningioma_tumor: 4.3478260869565215 %
Accuracy of no_tumor: 65.71428571428571 %
Accuracy of pituitary_tumor: 55.4054054054054 %
Epoch: 1 Train Loss: tensor(0.7206) Train Accuracy: 72.65166340508806 Test Accuracy: 45.57823129251701
Accuracy of meningioma_tumor: 46.08695652173913 %
Accuracy of no_tumor: 39.04761904761905 %
Accuracy of pituitary_tumor: 54.054054054054056 %
Epoch: 2 Train Loss: tensor(0.5652) Train Accuracy: 76.85909980430529 Test Accuracy: 46.93877551020408
Accuracy of meningioma_tumor: 22.608695652173914 %
Accuracy of no_tumor: 72.38095238095238 %
Accuracy of pituitary_tumor: 48.648648648648646 %
Epoch: 3 Train Loss: tensor(0.5184) Train Accuracy: 80.72407045009784 Test Accuracy: 55.78231292517007
Accuracy of meningioma_tumor: 59.130434782608695 %
Accuracy of no_tumor: 66.66666666666667 %
Accuracy of pituitary_tumor: 35.13513513513514 %
Epoch: 4 Train Loss: 

In [ ]:
#graphs
import numpy as np
import matplotlib.pyplot as plt

epochs = range(1,16)
plt.plot(epochs, loss_train, 'g')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('loss.png')
plt.show()

plt.plot(epochs, train_acc, 'g', label='Training accuracy')
plt.plot(epochs, test_acc, 'b', label='Test accuracy')
plt.title('Training and test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy in %')
plt.legend()
plt.savefig('accuracy.png')
plt.show()